# Packages

In [1]:
#General
import matplotlib
import math
import pandas

#Sourcing Tweets
import tweepy 
import datetime

import snscrape.modules.twitter as sntwitter
import itertools

#Preprocessing
import nltk

nltk.download("stopwords")
nltk.download('wordnet')
from nltk.corpus import stopwords

#Sentiment analysis
from textblob import Word, TextBlob

from transformers import pipeline

#Plotting Sentiment
import folium


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nicogouveia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nicogouveia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#  Sourcing tweets 


In [2]:
#Get API keys & tokens from local computer

#TODO add in images to jupyter notebook that show .txt file formatting with hidden tokens

with open('Geo tweet sentiment keysntokens.txt') as file:
    text = file.read()
    split = text.rsplit(':')
    
    consumer_key = split[1]
    consumer_secret = split[3]
    access_token = split[5]
    access_secret = split[7]
    Bearertok = split[9]


In [3]:
#Generating API

auth = tweepy.OAuth1UserHandler(
   consumer_key, consumer_secret, access_token, access_secret)

api = tweepy.API(auth,wait_on_rate_limit=True)

#testing
public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)
    

Zooming in to see the detail around the Snowbird ski area of UT.  I usually turn off my music before recording, but… https://t.co/X2qG6wSHWp
3/3
In the textureless version you can more clearly see were the trees were cleared for ski trails. https://t.co/Qzfd3E4NSR
2/3 
Even though the basemap in the textured LiDAR map is exactly the same, the green areas with trees looks like a… https://t.co/aQrtepnX16
1/3 
Several flavors of Telluride CO today.  3 versions of a 1955 USGS map and a 1904 one.  The 1955 is ground only… https://t.co/2J7xsgnkGD
Anyone know if you can collapse symbology en masse in ArcGIS Pro?  In ArcMap, I just select everything and hit the… https://t.co/xTU8Kduj51
RT @FalaciaDelDia: Buen día, mundo.

Paso a notificarles que ya se encuentra adornando mi pared esta preciosidad de mapa geológico de Sudam…
Geologic Map of Taiwan - Geologic Survey of Taiwan - 1953

3D render

#geology #geography #cartography #GIS #b3d… https://t.co/hwEOHa2zPD
#Borneo #geology #GIS #geography #

In [4]:

# Garbage function to extract tweet - I can't figure out how to extract tweets by location!

def get_tweets(query="transit", num_tweets= 1):
    
    data =  []
    i  =  1
    
    #Does geocode use these?
    #latitude = 44.052071
    #longitude = -123.086754
    #radius = 10
    
    while i <= num_tweets:
           
                #Garbage Query
                
            tweet = api.search_tweets(              q=query,
                                                    lang='en',
                                                    result_type= 'mixed',
                                                    tweet_mode="extended",
                #This is what I can't figure out -> geocode= '01241e9666cacdd2',
                                                    count=1)
            
            for item in tweet:
                
                mined = {
                    'tweet_id':        item.id,
                    'name':            item.user.name,
                    'screen_name':     item.user.screen_name,
                    'retweet_count':   item.retweet_count,
                    'text':            item.full_text,
                    'mined_at':        datetime.datetime.now(),
                    'created_at':      item.created_at,
                    'favourite_count': item.favorite_count,
                    'hashtags':        item.entities['hashtags'],
                    'status_count':    item.user.statuses_count,
                    'location':        item.place,
                    'source_device':   item.source
                }
                
                try:
                    mined['retweet_text'] = item.retweeted_status.full_text
                except:
                    mined['retweet_text'] = 'None'
                try:
                    mined['quote_text'] = item.quoted_status.full_text
                    mined['quote_screen_name'] = status.quoted_status.user.screen_name
                except:
                    mined['quote_text'] = 'None'
                    mined['quote_screen_name'] = 'None'
                data.append(mined)
                
            i+= 1
            
    return data 

tweet = get_tweets(query='University', num_tweets = 1)
tweet

# I can get tweets but can't source by location with Twitter API

[{'tweet_id': 1500308930970353667,
  'name': 'Narendra Modi',
  'screen_name': 'narendramodi',
  'retweet_count': 3638,
  'text': 'Leaving for Pune where I will be taking part in various programmes including the inauguration of the Metro Rail Project, laying the foundation stone for various development works and attending the Golden Jubilee celebrations of Symbiosis University. https://t.co/EzGbRJ36or',
  'mined_at': datetime.datetime(2022, 3, 6, 22, 12, 26, 26118),
  'created_at': datetime.datetime(2022, 3, 6, 3, 14, 58, tzinfo=datetime.timezone.utc),
  'favourite_count': 23763,
  'hashtags': [],
  'status_count': 32179,
  'location': None,
  'source_device': 'Twitter for iPhone',
  'retweet_text': 'None',
  'quote_text': 'None',
  'quote_screen_name': 'None'}]

In [8]:
#Derive place id

places = api.search_geo(query="Eugene", granularity="city")
place_id = places[0].id
place_id #id='01241e9666cacdd2'


'01241e9666cacdd2'

In [37]:
# our search term, using syntax for Twitter's Advanced Search
query_term = 'transit'
location = " place:01241e9666cacdd2" #Eugene
lang = " lang:en"
search = query_term + location +lang
K = 10

# the scraped tweets, this is a generator
scraped_tweets = sntwitter.TwitterSearchScraper(search).get_items()

# slicing the generator to keep only the first K tweets
sliced_scraped_tweets = itertools.islice(scraped_tweets, K)

# convert to a DataFrame and keep only relevant columns
df = pandas.DataFrame(sliced_scraped_tweets)[['date',"user",'content']]

We'll use the snscraper tool instead:

In [38]:
df

,date,user,content
0,2022-01-10 20:43:48+00:00,"{'username': 'subumbral', 'id': 256319644, 'di...","@RetiredDent I know it's airborne, but public ..."
1,2021-10-23 02:52:36+00:00,"{'username': 'cptarkennedy', 'id': 620283234, ...",AI and human transit will allow for green urba...
2,2021-10-04 03:50:23+00:00,"{'username': 'subumbral', 'id': 256319644, 'di...",@spindlypete Sic transit
3,2021-08-12 22:14:30+00:00,"{'username': 'groovybels', 'id': 2744006288, '...","Venus about to transit my 4H where my sun, moo..."
4,2021-08-11 22:21:40+00:00,"{'username': 'groovybels', 'id': 2744006288, '...",how fitting of a libra moon to buy jewelry and...
5,2021-08-05 00:18:26+00:00,"{'username': 'groovybels', 'id': 2744006288, '...",Pluto transiting my 7H Neptune in cap today an...
6,2021-08-04 18:14:51+00:00,"{'username': 'GoDucksJosh', 'id': 151119910, '...",Hey @FedEx @FedExHelp is this normal? How many...
7,2021-08-02 23:09:05+00:00,"{'username': 'GoDucksJosh', 'id': 151119910, '...",@DTFinnegan I thought I was good and it avoide...
8,2021-07-14 02:22:42+00:00,"{'username': 'subumbral', 'id': 256319644, 'di...",@KaylaChowShow On a day trip to another city b...
9,2021-06-26 19:39:31+00:00,"{'username': 'JamesLLlamas', 'id': 3308309264,...",Transit Oriented Tubing! https://t.co/M4P23gfTMI


# Preprocessing

In [39]:
custom_stopwords= ['RT'] #if needed, add more depending on search term

def preprocess_tweet(tweet,custom_stopwords):
    preprocessed_tweet = tweet
    preprocessed_tweet.replace('[^\w\s]','')
    preprocessed_tweet = ' '.join(word for word in preprocessed_tweet.split() if word not in stopwords.words('english'))
    preprocessed_tweet = ' '.join(word for word in preprocessed_tweet.split() if word not in custom_stopwords)
    preprocessed_tweet = ' '.join(Word(word).lemmatize() for word in preprocessed_tweet.split())
    return(preprocessed_tweet)

In [40]:
df["Processed Tweet"] = df ["content"].apply(lambda x: preprocess_tweet(x, custom_stopwords))
df

,date,user,content,Processed Tweet
0,2022-01-10 20:43:48+00:00,"{'username': 'subumbral', 'id': 256319644, 'di...","@RetiredDent I know it's airborne, but public ...","@RetiredDent I know airborne, public transit p..."
1,2021-10-23 02:52:36+00:00,"{'username': 'cptarkennedy', 'id': 620283234, ...",AI and human transit will allow for green urba...,AI human transit allow green urbanization wild...
2,2021-10-04 03:50:23+00:00,"{'username': 'subumbral', 'id': 256319644, 'di...",@spindlypete Sic transit,@spindlypete Sic transit
3,2021-08-12 22:14:30+00:00,"{'username': 'groovybels', 'id': 2744006288, '...","Venus about to transit my 4H where my sun, moo...","Venus transit 4H sun, moon Mercury reside two ..."
4,2021-08-11 22:21:40+00:00,"{'username': 'groovybels', 'id': 2744006288, '...",how fitting of a libra moon to buy jewelry and...,fitting libra moon buy jewelry get nail done l...
5,2021-08-05 00:18:26+00:00,"{'username': 'groovybels', 'id': 2744006288, '...",Pluto transiting my 7H Neptune in cap today an...,Pluto transiting 7H Neptune cap today boy tran...
6,2021-08-04 18:14:51+00:00,"{'username': 'GoDucksJosh', 'id': 151119910, '...",Hey @FedEx @FedExHelp is this normal? How many...,Hey @FedEx @FedExHelp normal? How many day pac...
7,2021-08-02 23:09:05+00:00,"{'username': 'GoDucksJosh', 'id': 151119910, '...",@DTFinnegan I thought I was good and it avoide...,@DTFinnegan I thought I good avoided hellhole ...
8,2021-07-14 02:22:42+00:00,"{'username': 'subumbral', 'id': 256319644, 'di...",@KaylaChowShow On a day trip to another city b...,@KaylaChowShow On day trip another city mass t...
9,2021-06-26 19:39:31+00:00,"{'username': 'JamesLLlamas', 'id': 3308309264,...",Transit Oriented Tubing! https://t.co/M4P23gfTMI,Transit Oriented Tubing! https://t.co/M4P23gfTMI


# Sentiment Analysis (polarity & subjectivity)

In [41]:

# Uses pytorch, huggingface and a default model(?) to determine polarity and add it to df

classifier = pipeline("sentiment-analysis")

df['pipeline label'] = df["Processed Tweet"].apply(lambda x: classifier(x)[0]['label'])
df['pipeline score'] = df["Processed Tweet"].apply(lambda x: classifier(x)[0]['score'])

df.loc[df['pipeline label'] == 'NEGATIVE', 'pipeline score'] = df['pipeline score']*-1
df

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


,date,user,content,Processed Tweet,pipeline label,pipeline score
0,2022-01-10 20:43:48+00:00,"{'username': 'subumbral', 'id': 256319644, 'di...","@RetiredDent I know it's airborne, but public ...","@RetiredDent I know airborne, public transit p...",NEGATIVE,-0.999802
1,2021-10-23 02:52:36+00:00,"{'username': 'cptarkennedy', 'id': 620283234, ...",AI and human transit will allow for green urba...,AI human transit allow green urbanization wild...,POSITIVE,0.982988
2,2021-10-04 03:50:23+00:00,"{'username': 'subumbral', 'id': 256319644, 'di...",@spindlypete Sic transit,@spindlypete Sic transit,NEGATIVE,-0.977179
3,2021-08-12 22:14:30+00:00,"{'username': 'groovybels', 'id': 2744006288, '...","Venus about to transit my 4H where my sun, moo...","Venus transit 4H sun, moon Mercury reside two ...",NEGATIVE,-0.998847
4,2021-08-11 22:21:40+00:00,"{'username': 'groovybels', 'id': 2744006288, '...",how fitting of a libra moon to buy jewelry and...,fitting libra moon buy jewelry get nail done l...,POSITIVE,0.960011
5,2021-08-05 00:18:26+00:00,"{'username': 'groovybels', 'id': 2744006288, '...",Pluto transiting my 7H Neptune in cap today an...,Pluto transiting 7H Neptune cap today boy tran...,POSITIVE,0.835144
6,2021-08-04 18:14:51+00:00,"{'username': 'GoDucksJosh', 'id': 151119910, '...",Hey @FedEx @FedExHelp is this normal? How many...,Hey @FedEx @FedExHelp normal? How many day pac...,NEGATIVE,-0.996744
7,2021-08-02 23:09:05+00:00,"{'username': 'GoDucksJosh', 'id': 151119910, '...",@DTFinnegan I thought I was good and it avoide...,@DTFinnegan I thought I good avoided hellhole ...,NEGATIVE,-0.957317
8,2021-07-14 02:22:42+00:00,"{'username': 'subumbral', 'id': 256319644, 'di...",@KaylaChowShow On a day trip to another city b...,@KaylaChowShow On day trip another city mass t...,NEGATIVE,-0.937028
9,2021-06-26 19:39:31+00:00,"{'username': 'JamesLLlamas', 'id': 3308309264,...",Transit Oriented Tubing! https://t.co/M4P23gfTMI,Transit Oriented Tubing! https://t.co/M4P23gfTMI,NEGATIVE,-0.990848


In [42]:
# Uses TextBlob to derive sentiment of each tweet and add it to the df

df['Polarity'] = df["Processed Tweet"].apply(lambda x: TextBlob(x).sentiment[0])
df['Subjectivity'] = df["Processed Tweet"].apply(lambda x: TextBlob(x).sentiment[1])

TODO: expand and improve this using machine learning! 

I don't really understand how TextBlob is deriving these values. They also do not seem that accurate, however:
- Sentiment polarity ranges from -1 (bad opinion) to 1 (good opinion).
- Sentiment subjectivty ranges from 0 (very objective) to 1 (very subjective).

In [43]:
df

,date,user,content,Processed Tweet,pipeline label,pipeline score,Polarity,Subjectivity
0,2022-01-10 20:43:48+00:00,"{'username': 'subumbral', 'id': 256319644, 'di...","@RetiredDent I know it's airborne, but public ...","@RetiredDent I know airborne, public transit p...",NEGATIVE,-0.999802,0.000000,0.033333
1,2021-10-23 02:52:36+00:00,"{'username': 'cptarkennedy', 'id': 620283234, ...",AI and human transit will allow for green urba...,AI human transit allow green urbanization wild...,POSITIVE,0.982988,0.036061,0.497879
2,2021-10-04 03:50:23+00:00,"{'username': 'subumbral', 'id': 256319644, 'di...",@spindlypete Sic transit,@spindlypete Sic transit,NEGATIVE,-0.977179,0.000000,0.000000
3,2021-08-12 22:14:30+00:00,"{'username': 'groovybels', 'id': 2744006288, '...","Venus about to transit my 4H where my sun, moo...","Venus transit 4H sun, moon Mercury reside two ...",NEGATIVE,-0.998847,-0.300000,0.450000
4,2021-08-11 22:21:40+00:00,"{'username': 'groovybels', 'id': 2744006288, '...",how fitting of a libra moon to buy jewelry and...,fitting libra moon buy jewelry get nail done l...,POSITIVE,0.960011,0.500000,0.500000
5,2021-08-05 00:18:26+00:00,"{'username': 'groovybels', 'id': 2744006288, '...",Pluto transiting my 7H Neptune in cap today an...,Pluto transiting 7H Neptune cap today boy tran...,POSITIVE,0.835144,0.000000,0.000000
6,2021-08-04 18:14:51+00:00,"{'username': 'GoDucksJosh', 'id': 151119910, '...",Hey @FedEx @FedExHelp is this normal? How many...,Hey @FedEx @FedExHelp normal? How many day pac...,NEGATIVE,-0.996744,0.183333,0.516667
7,2021-08-02 23:09:05+00:00,"{'username': 'GoDucksJosh', 'id': 151119910, '...",@DTFinnegan I thought I was good and it avoide...,@DTFinnegan I thought I good avoided hellhole ...,NEGATIVE,-0.957317,0.700000,0.600000
8,2021-07-14 02:22:42+00:00,"{'username': 'subumbral', 'id': 256319644, 'di...",@KaylaChowShow On a day trip to another city b...,@KaylaChowShow On day trip another city mass t...,NEGATIVE,-0.937028,0.000000,0.000000
9,2021-06-26 19:39:31+00:00,"{'username': 'JamesLLlamas', 'id': 3308309264,...",Transit Oriented Tubing! https://t.co/M4P23gfTMI,Transit Oriented Tubing! https://t.co/M4P23gfTMI,NEGATIVE,-0.990848,0.000000,0.000000


In [45]:
#Calculate avg sentiment for the entire df (city)
avg_sentiment =df.mean(numeric_only = True)
# avg_polarity = avg_sentiment[1]
# avg_subjectivity = avg_sentiment[2]

print(avg_sentiment)


pipeline score   -0.407962
Polarity          0.111939
Subjectivity      0.259788
dtype: float64


### Now we create a pipeline

In [107]:
city_IDs = {}

class city:
    
    #City creater method - gets place ID using Twitter API and name
    def __init__(self, name): 
        self.name = name
        
        self.search = None
    
        self.polarity = None
        
        self.pytorch_polarity = None
    
        self.subjectivty = None 
        
        if name in city_IDs:
            self.place_id = city_IDs[name] 
        else:
            city_IDs[name]= api.search_geo(query=name, granularity="city")[0].id #requires api
            self.place_id =city_IDs[name]
            
        self.coords = None 
        
        self.df = None
    
    #Creates df with tweets
    def derive_tweets(self, query_term, num_tweets=10, language='en'):
        
        self.search = query_term
        
        # our search term, using syntax for Twitter's Advanced Search
        location = " place:"+ self.place_id 
        lang = " lang:" +language
        search_phrase = query_term + location +lang
       
        # the scraped tweets, this is a generator
        scraped_tweets = sntwitter.TwitterSearchScraper(search_phrase).get_items()

        # slicing the generator to keep only the first num_tweets tweets
        sliced_scraped_tweets = itertools.islice(scraped_tweets, num_tweets)

        # convert to a DataFrame and keep only relevant columns
        self.df = pandas.DataFrame(sliced_scraped_tweets)[['date','coordinates',"user",'content']]
        
        #all coords are the same
        self.coords = self.df.coordinates[0]
    
    #Updates df with formatted tweets
    def preprocess_tweets(self):
        
        self.df["Processed Tweet"] = self.df["content"].apply(lambda x: preprocess_tweet(x, custom_stopwords))

    #Runs sentiment analysis on df and updates city object    
    def derive_sentiment(self):
        
        
        self.df['Polarity'] = self.df["Processed Tweet"].apply(lambda x: TextBlob(x).sentiment[0])
        self.df['Subjectivity'] = self.df["Processed Tweet"].apply(lambda x: TextBlob(x).sentiment[1])
        
        #TODO use this other polarity later
        self.df['pipeline label'] = self.df["Processed Tweet"].apply(lambda x: classifier(x)[0]['label'])
        self.df['pipeline score'] = self.df["Processed Tweet"].apply(lambda x: classifier(x)[0]['score'])
        self.df.loc[self.df['pipeline label'] == 'NEGATIVE', 'pipeline score'] = self.df['pipeline score']*-1
        
        avg_sentiment =self.df.mean(numeric_only = True)
        self.polarity = avg_sentiment[0]
        self.subjectivity = avg_sentiment[1]
        self.pytorch_polarity = avg_sentiment[2]#TODO use subjectivity later
    

In [108]:
#Derives opinion of query term for the city object

#note - each city object can only have one search term associated with it
def evaluate_city(city, query, num_tweets):
    
    city.derive_tweets(query, num_tweets)
    city.preprocess_tweets()
    city.derive_sentiment()

### Here we create 3 city objects and feed them the search term "Oregon ducks" to evaluate

In [110]:
sea = city("Seattle")
evaluate_city(sea, 'Oregon ducks', 10)

eug = city("Eugene")
evaluate_city(eug, 'Oregon ducks', 10)

corvo = city("Corvallis")
evaluate_city(corvo, 'Oregon ducks', 10)


In [111]:
print("Seattle opinion of 'ducks': " + str(sea.polarity))
print("Corvallis opinion of 'ducks': " + str(corvo.polarity))
print("Eugene opinion of 'ducks': " + str(eug.polarity))

Seattle opinion of 'ducks': 0.008518518518518514
Corvallis opinion of 'ducks': -0.044666666666666674
Eugene opinion of 'ducks': 0.20697159090909087


## Visualization

In [90]:
#determine icon color and symbol via sentiment (polarity)

def sent_gradient(polarity):
    if (polarity < -.50) and (polarity > -1):
        return "darkred"
    if polarity < -.2:
        return "red"
    if polarity < -.0:
        return "lightred"
    if (polarity == 0):
        return "beige"
    if polarity < .2:
        return "lightgreen"
    if polarity < .50:
        return "green"
    if polarity < 1:
        return "darkgreen"
    else:
        return None

def sent_icon(polarity):
    if (polarity < 0) and (polarity > -1):
        return "glyphicon-thumbs-down"
    if (polarity < 1) and (polarity > 0):
        return "glyphicon-thumbs-up"
    if (polarity == 0):
        return "glyphicon-resize-horizontal"
    else:
        return None

In [91]:

#create test map
watercolor = folium.Map(location=[ 45.5236,-122.6750],
                        tiles="Stamen Watercolor",
                        zoom_start=5)

#create marker from evaluated city and add marker to map
def marker_maker(city, map_name,mode="polarity"):
    
    if (mode =="pytorch"):
        polarity = city.pytorch_polarity
    if (mode == "combo"):
        polarity = (city.pytorch_polarity + city.polarity)/2
    if (mode == "polarity"):
        polarity = city.polarity
    location = [ (city.coords['latitude']),(city.coords['longitude']) ]
    detail = city.name + ' polarity for the term "'+ city.search + '" is: ' + str(polarity)

    folium.Marker(location,
                  popup= detail, 
                  #TODO add in more detailed popups
                  tooltip=None, 
                  icon= folium.Icon(icon= sent_icon(polarity), color = sent_gradient(polarity)),
                  #TODO add in more detailed icons using folium.plugins.BeautifyIcon
                  draggable=False,).add_to(map_name)

### Here we are plotting the markers for each city object, then displaying the map

In [126]:
marker_maker(eug, watercolor)
marker_maker(corvo, watercolor)
marker_maker(sea, watercolor)
watercolor


# Putting it all together


In [123]:
query =  "alex jones"
        #"alex jones" -polarity
        #"joe rogan"
        #"sushi" -combo
num_tweets = 50 #determines accuracy

rogan = folium.Map(location=[ 45.5236,-80.6750],
                        tiles="Stamen Toner",
                        zoom_start=3)

city_list = ['Vancouver',
             #'Boise',
             'Seattle',
             'Portland',
             #'Corvallis',
             'Oakland',
             #'Bend',
             #'Astoria',
             #'Springfield OR',
             'Los Angeles',
             'Chicago',
             'New York',
             'Boston',
             'Washington',
             'Miami',
             'Denver']


def analyze_city(city_name, query, map_name): 

    print("Scraping "+ city_name + " tweets")
    city_obj = city(city_name) #75 new cities every 15 minutes
    
    print("Calculating "+ city_name + " sentiment")
    evaluate_city(city_obj,query, num_tweets)
    
    print("Prepping "+ city_name + " marker \n")
    marker_maker(city_obj, map_name, "combo")

    
    
for item in city_list:
    try:
        analyze_city(item, query, rogan)
    except:
        print("SNScraper error - skipping city \n")
        next
    
rogan




Scraping Vancouver tweets
Calculating Vancouver sentiment
Prepping Vancouver marker 

Scraping Seattle tweets
Calculating Seattle sentiment
Prepping Seattle marker 

Scraping Portland tweets
Calculating Portland sentiment
Prepping Portland marker 

Scraping Oakland tweets
Calculating Oakland sentiment
Prepping Oakland marker 

Scraping Los Angeles tweets
Calculating Los Angeles sentiment
Prepping Los Angeles marker 

Scraping Chicago tweets
Calculating Chicago sentiment
SNScraper error - skipping city 

Scraping New York tweets
Calculating New York sentiment
Prepping New York marker 

Scraping Boston tweets
Calculating Boston sentiment
Prepping Boston marker 

Scraping Washington tweets
Calculating Washington sentiment
Prepping Washington marker 

Scraping Miami tweets
Calculating Miami sentiment
Prepping Miami marker 

Scraping Denver tweets
Calculating Denver sentiment
Prepping Denver marker 

